In [ ]:
import sys
sys.path.insert(0, '/Users/zacksiegfried/Documents/methylspan')
import MethylFormatting
import MethylMapping


all_files = MethylFormatting.pullMethylMetaData('pancreas')

# selecting only methylation assay 450 files
all_files2 = []
for i in all_files:
    if i['platform'] == 'Illumina Human Methylation 450':
        all_files2.append(i)

# not gaureteed to return same number of cases, (remove file_id from sample data if missing from meta data)
# missing values dropped anyways in next steps
sample_data = MethylFormatting.methylDataFormat(all_files2, 50)
meta_data = MethylFormatting.metaDataFormat(sample_data, all_files2)

map_test = MethylMapping.methylMap450(sample_data)

In [3]:
#map_test.to_csv('data/map_test.csv')

from collections import Counter
gene_dict = Counter(map_test.columns)
#print(gene_dict)

In [ ]:
### AVERAGE GENE REGION METHOD
import MethylMapping

sample_average_data = MethylMapping.averageGene(map_test, 'MGMT')

In [ ]:
### FULL GENE REGION METHOD
import MethylMapping

sample_full_data = MethylMapping.fullGene(map_test, 'MGMT')

In [ ]:
### FORMATTING FOR SURVIVAL

import pandas as pd
from lifelines import CoxPHFitter
from lifelines.utils import k_fold_cross_validation

def mappingSurvivalFormat(sample_data, meta_data):
    
    meta_data.index.name = 'id'
    sample_data.index.name = 'id'
    sample_data = sample_data.dropna(axis=1)

    # merges dfs and formats 
    full_data = pd.merge(meta_data, sample_data, on = 'id')
    full_data.replace(['Dead', 'Alive'],[1, 0], inplace=True)
    full_data = full_data[full_data['days_to_death'].notna()]

    print("Final number of cases used: " + str(full_data.shape[0]))

    return(full_data)


full_data = mappingSurvivalFormat(sample_average_data, meta_data)

In [ ]:
### SURVIVAL ANALYSIS EXECUTION & TESTING

# model trained on complete data set 
cph = CoxPHFitter()
cph.fit(full_data, duration_col='days_to_death', event_col='vital_status')
print(cph.confidence_intervals_)

# Concordance index (will remodel cph.)
scores = k_fold_cross_validation(cph, full_data, 'days_to_death', event_col='vital_status', k=5, scoring_method="concordance_index")
print(scores)